In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import os

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [ ]:
import os
if IN_COLAB:
    print("running in google colab")
    from google.colab import drive
    drive.mount("/content/drive")
    os.chdir("/content/drive/My Drive/seminar_dl_workspace")
    print("switched workspace:", os.getcwd())

from create_visualizations import get_filenames_in_dir

In [ ]:
from IPython.display import SVG, display
def show_svg(filename):
    display(SVG(filename=filename))

boilerplate done...

In [ ]:
batch_sizes = [4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048]
momentums = [0, 0.9]

def get_batch_size_from_experiment_name(name):
    matches = re.search(r"([0-9]*\.?[0-9]+)_(\d+)\.csv$", name)
    momentum = float(matches.group(1))
    batch_size = int(matches.group(2))
    return batch_size, momentum

def get_latest_experiment_df(csv_path):
    experiment_stats = pd.read_csv(csv_path)
    experiment_stats_grouped = experiment_stats.groupby("experiment_id")
    latest_training_session = experiment_stats_grouped.get_group(experiment_stats.experiment_id.max())
    if len(experiment_stats) < 20:
        print(f"WARNING: file {csv_path} has only {len(experiment_stats)} epochs but should have 20")
    elif len(experiment_stats) > 20 and len(experiment_stats) % 20 == 0:
        print(f"INFO: file {csv_path} has {experiment_stats_grouped.ngroups} training sessions. Using latest training session with {len(latest_training_session)} epochs and experiment_id {latest_training_session.iloc[0]['experiment_id']}")
    return latest_training_session

In [ ]:
dims = (len(batch_sizes), len(momentums))
results_test_acc = np.zeros(dims)
results_train_acc = np.zeros(dims)
results_test_avg_loss = np.zeros(dims)
results_train_avg_loss = np.zeros(dims)

experiments_stats_filenames = get_filenames_in_dir("experiments_stats", lambda x: "RMSprop_" in x)
print(f"found {len(experiments_stats_filenames)} experiments for RMSprop_")
for filename in experiments_stats_filenames:
    batch_size, momentum = get_batch_size_from_experiment_name(filename)
    i = batch_sizes.index(batch_size)
    k = momentums.index(momentum)
    df = get_latest_experiment_df(os.path.join("experiments_stats" , filename))
    results_test_acc[i, k] = df["test_acc"].max()
    results_train_acc[i, k] = df["train_acc"].max()
    results_test_avg_loss[i, k] = df["test_avg_loss"].min()
    results_train_avg_loss[i, k] = df["train_avg_loss"].min()

max_train_acc = results_train_acc.max()
max_test_acc = results_test_acc.max()
min_train_loss = results_train_avg_loss.min()
min_test_loss = results_test_avg_loss.min()
print(f"max train accuracy: batch_size={batch_sizes[np.where(results_train_acc == max_train_acc)[0][0]]} momentum={momentums[np.where(results_train_acc == max_train_acc)[1][0]]} --> {max_train_acc}")
print(f"max test accuracy: batch_size={batch_sizes[np.where(results_test_acc == max_test_acc)[0][0]]} momentum={momentums[np.where(results_test_acc == max_test_acc)[1][0]]} --> {max_test_acc}")
print(f"min train avg loss: batch_size={batch_sizes[np.where(results_train_avg_loss == min_train_loss)[0][0]]} momentum={momentums[np.where(results_train_avg_loss == min_train_loss)[1][0]]} --> {min_train_loss}")
print(f"min test avg loss: batch_size={batch_sizes[np.where(results_test_avg_loss == min_test_loss)[0][0]]} momentum={momentums[np.where(results_test_avg_loss == min_test_loss)[1][0]]} --> {min_test_loss}")

In [ ]:
def draw_subplot_bar(ax, data, title, ylabel="", ylim=None):
    bar_width = 0.38
    r1 = [x for x in np.arange(len(data[:, 0]))]
    r2 = [x + bar_width for x in r1]
    x_labels = list(map(lambda x: str(x), batch_sizes))
    ax.bar(r1, data[:, 0], width=bar_width, label="gamma=0")
    ax.bar(r2, data[:, 1], width=bar_width, color="orange", label="gamma=0.9")

    ax.set_title(title)
    ax.set_ylabel(ylabel)
    ax.set_xlabel("batch size")
    ax.set_xticks([r + bar_width / 2 for r in r1])
    ax.set_xticklabels(x_labels)
    ax.legend()

    bar_label_y_padding = 0
    bar_label_y_pos = 0
    if ylim is not None:
        ax.set_ylim(ylim)
        bar_label_y_padding = (ylim[1] - ylim[0]) * 0.025
        bar_label_y_pos = ylim[0]
    for i, v in enumerate(data[:, 0]):
        ax.text(i - bar_width / 2, v + bar_label_y_padding, round(v, 3), fontsize=11, color="black")
    for i, v in enumerate(data[:, 1]):
        ax.text(i + bar_width / 2, v, round(v, 3), fontsize=11, color="black")

Plot avg losses and accuracies for different batch sizes

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(28, 15), gridspec_kw=dict(hspace=0.35))
fig.suptitle("Rmsprop for different batch sizes after 20 epochs")

draw_subplot_bar(axs[0, 0], results_train_acc, "Train Accuracy for different batch sizes", ylabel="accuracy", ylim=(0.8, 1.0))
draw_subplot_bar(axs[0, 1], results_test_acc, "Test Accuracy for different batch sizes", ylabel="accuracy", ylim=(0.65, 0.85))
draw_subplot_bar(axs[1, 0], results_train_avg_loss, "Train avg loss for different batch sizes", ylabel="loss", ylim=(0.0, 0.55))
draw_subplot_bar(axs[1, 1], results_test_avg_loss, "Test avg loss for different batch sizes", ylabel="loss", ylim=(0.4, 1.0))

plt.savefig("visualizations/rmsprop_batch_sizes.png")